In [44]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import neighbors
import pandas as pd
from random import uniform

from sklearn.datasets import load_iris
from sklearn import tree

import sys
sys.path.append('../')
from database.models import get_db_session, NumericCluster, Incidencia, AccidentDBScan

In [45]:
session = get_db_session('sqlite:///../incidences.db')

In [46]:
def tf_nivel_medio(a):
    n = 0
    if a == "Rojo":
        n = 5
    elif a == "Blanco":
        n = 1
    else:
        n = 10

    return n

def tf_carretera(a):
    n = 0

    if "BI-" in a:
        n = 1
    elif "N-" in a:
        n = 2
    else:
        n = 3

    return n

def tf_causa_ppal(a):
    n = 0

    if a == "Alcance":
        n = 1
    elif a == "Salida":
        n = 3
    else :
        n = 4

    return n

def get_useful_data(accident):
    res = []
    res.append(tf_causa_ppal(accident.causa))
    res.append(tf_nivel_medio(accident.nivel))
    res.append(tf_carretera(accident.carretera))
    
    return res

In [47]:
# Get the clusters, extract their data, obtain the causes, print the feature names
# and the target
accidents = session.query(Incidencia).filter(Incidencia.tipo == 'Accidente').all()
clusters = session.query(AccidentDBScan.num_cluster).all()
targets = [cluster[0] for cluster in clusters]
data = [get_useful_data(accident) for accident in accidents]
feature_names = ['Causa', 'Nivel del accidente', 'Carretera']
target_names = [str(target) for target in targets]

In [54]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(data, targets)

from sklearn.externals.six import StringIO  
import pydot

dot_data = StringIO() 
tree.export_graphviz(clf, out_file=dot_data,  
                         feature_names=feature_names,  
                         #class_names=target_names,  
                         filled=True, rounded=True,  
                         special_characters=True)

## convert the logics into graph
graph = pydot.graph_from_dot_data(dot_data.getvalue())  

## This will plot decision tree in pdf file
graph.write_png(path="iris.png") 


True